## 1. Install and import dependencies

In [27]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [28]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 2. Instantiate the model

In [4]:
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\ishik\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ishik\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## 3. Encode and Calculate Sentiment

In [11]:
tokens= tokenizer.encode('This is amazing , I loved it!', return_tensors='pt')
tokens[0]

tensor([  101, 10372, 10127, 39854,   117,   151, 46747, 10197,   106,   102])

In [14]:
tokenizer.decode(tokens[0])

'[CLS] this is amazing, i loved it! [SEP]'

In [12]:
result=model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4866, -2.8335, -1.2113,  1.3646,  4.1660]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
#gives sentiment score from 1 to 5
int(torch.argmax(result.logits))+1

5

## 4. Collect reviews

In [15]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [17]:
reviews

['Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.',
 'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.',
 'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.",
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coff

## 5. Load Reviews into DataFrame and Score

In [18]:
import pandas as pd
import numpy as np


In [19]:
df=pd.DataFrame(np.array(reviews),columns=['review'])

In [20]:
df.head()

,review
0,Some of the best Milkshakes me and my daughter...
1,Great place with delicious food and friendly s...
2,Great food amazing coffee and tea. Short walk ...
3,It was ok. Had coffee with my friends. I'm new...
4,Ricotta hot cakes! These were so yummy. I ate ...


In [21]:
def sentiment_score(review):
    tokens=tokenizer.encode(review,return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1

In [22]:
sentiment_score(df['review'].iloc[1])

5

In [23]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [24]:
df

,review,sentiment
0,Some of the best Milkshakes me and my daughter...,5
1,Great place with delicious food and friendly s...,5
2,Great food amazing coffee and tea. Short walk ...,5
3,It was ok. Had coffee with my friends. I'm new...,3
4,Ricotta hot cakes! These were so yummy. I ate ...,5
5,Great staff and food. Must try is the pan fri...,5
6,We came for brunch twice in our week-long visi...,4
7,I came to Social brew cafe for brunch while ex...,5
8,It was ok. The coffee wasn't the best but it w...,3
9,Great coffee and vibe. That's all you need. C...,5
